# (Quantum) Neural Networks

This notebook demonstrates the different generic (quantum) neural network implementations provided in Qiskit Machine Learning.
The networks are meant as application-anostic computational units that can be used in different use cases. 
Depending on the application, a particluar type of network might more or less suitable.
In the following, the different available networks will be discussed in more detail:

1. `NeuralNetwork`: The interface for neural networks.
2. `OpflowQNN`: A network based on the evaluation of quantum mechanical observables.
3. `TwoLayerQNN`: A special `OpflowQNN` implementation for convenience. 
3. `CircuitQNN`: A network based on the samples resulting from measuring a quantum circuit.

In [1]:
import numpy as np

from qiskit import Aer, QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit.opflow import StateFn, PauliSumOp, AerPauliExpectation, ListOp, Gradient
from qiskit.utils import QuantumInstance

In [2]:
# set method to calculcate expected values
expval = AerPauliExpectation()

# define gradient method
gradient = Gradient()

# define quantum instances (statevector and sample based)
qi_sv = QuantumInstance(Aer.get_backend('statevector_simulator'))
qi_qasm = QuantumInstance(Aer.get_backend('qasm_simulator'), shots=100) 

## 1. `NeuralNetwork`

The `NeuralNetwork` represents the interface for all neural networks available in Qiskit Machine Learning.
It just exposes a forward and a backward pass taking the data samples and trainable weights as input.
A `NeuralNetwork` does not contain any training capabilities, these are pushed to the actual algorithms / applications. Thus, a `NeuralNetwork` also does not store the values for trainable weights. In the following, different implementations of this interfaces are introduced.

Suppose a `NeuralNetwork` called `nn`.
Then, the `nn.forward(input, weights)` pass takes either flat inputs for the data and weights of size `nn.num_inputs` and `nn.num_weights`, respectively, or corresponding batches.

## 2. `OpflowQNN`

In [3]:
from qiskit_machine_learning.neural_networks import OpflowQNN

In [4]:
# construct parametrized circuit
params1 = [Parameter('input1'), Parameter('weight1')]
qc1 = QuantumCircuit(1)
qc1.h(0)
qc1.ry(params1[0], 0)
qc1.rx(params1[1], 0)
qc_sfn1 = StateFn(qc1)

# construct cost operator
H1 = StateFn(PauliSumOp.from_list([('Z', 1.0), ('X', 1.0)]))

# combine operator and circuit to objective function
op1 = ~H1 @ qc_sfn1
print(op1)

ComposedOp([
  OperatorMeasurement(1.0 * Z
  + 1.0 * X),
  CircuitStateFn(
       ┌───┐┌────────────┐┌─────────────┐
  q_0: ┤ H ├┤ RY(input1) ├┤ RX(weight1) ├
       └───┘└────────────┘└─────────────┘
  )
])


In [5]:
# construct OpflowQNN with the operator, the input parameters, the weight parameters, 
# the expected value, gradient, and quantum instance.
qnn1 = OpflowQNN(op1, [params1[0]], [params1[1]], expval, gradient, qi_sv)

In [6]:
# define (random) input and weights
input1 = np.random.rand(qnn1.num_inputs)
weights1 = np.random.rand(qnn1.num_weights)

In [7]:
# QNN forward pass
qnn1.forward(input1, weights1)

array(0.53161359)

In [8]:
# QNN backward pass
qnn1.backward(input1, weights1)

(array([-1.26545673]), array([0.13841381]))

<font color="red">INCLUDE BATCHES...</font>

Combining multiple observables in a `ListOp` also allows to create more complex QNNs

In [9]:
op2 = ListOp([op1, op1])
qnn2 = OpflowQNN(op2, [params1[0]], [params1[1]], expval, gradient, qi_sv)

In [10]:
# QNN forward pass
qnn2.forward(input1, weights1)

array([0.53161359, 0.53161359])

In [11]:
# QNN backward pass
qnn2.backward(input1, weights1)

(array([[-1.26545673],
        [-1.26545673]]),
 array([[0.13841381],
        [0.13841381]]))

## 3. `TwoLayerQNN`

The `TwoLayerQNN` is a special `OpflowQNN` on $n$ qubits that consists of first a feature map to insert data and second a variational form that is trained. The default observable is $Z^{\otimes n}$, i.e., parity.

In [12]:
from qiskit_machine_learning.neural_networks import TwoLayerQNN

In [13]:
# specify the number of qubits
num_qubits = 3

In [14]:
# specify the feature map
fm = ZZFeatureMap(num_qubits, reps=2)
print(fm.draw())

     ┌───┐┌─────────────┐                                               »
q_0: ┤ H ├┤ P(2.0*x[0]) ├──■────────────────────────────────────■────■──»
     ├───┤├─────────────┤┌─┴─┐┌──────────────────────────────┐┌─┴─┐  │  »
q_1: ┤ H ├┤ P(2.0*x[1]) ├┤ X ├┤ P(2.0*(π - x[0])*(π - x[1])) ├┤ X ├──┼──»
     ├───┤├─────────────┤└───┘└──────────────────────────────┘└───┘┌─┴─┐»
q_2: ┤ H ├┤ P(2.0*x[2]) ├──────────────────────────────────────────┤ X ├»
     └───┘└─────────────┘                                          └───┘»
«                                          ┌───┐»
«q_0: ──────────────────────────────────■──┤ H ├»
«                                       │  └───┘»
«q_1: ──────────────────────────────────┼────■──»
«     ┌──────────────────────────────┐┌─┴─┐┌─┴─┐»
«q_2: ┤ P(2.0*(π - x[0])*(π - x[2])) ├┤ X ├┤ X ├»
«     └──────────────────────────────┘└───┘└───┘»
«             ┌─────────────┐                                       »
«q_0: ────────┤ P(2.0*x[0]) ├─────────────────────────────────

In [15]:
# specify the variational form
var_form = RealAmplitudes(num_qubits, reps=1)
print(var_form.draw())

     ┌──────────┐          ┌──────────┐            
q_0: ┤ RY(θ[0]) ├──■────■──┤ RY(θ[3]) ├────────────
     ├──────────┤┌─┴─┐  │  └──────────┘┌──────────┐
q_1: ┤ RY(θ[1]) ├┤ X ├──┼───────■──────┤ RY(θ[4]) ├
     ├──────────┤└───┘┌─┴─┐   ┌─┴─┐    ├──────────┤
q_2: ┤ RY(θ[2]) ├─────┤ X ├───┤ X ├────┤ RY(θ[5]) ├
     └──────────┘     └───┘   └───┘    └──────────┘


In [16]:
# specify the observable
observable = PauliSumOp.from_list([('Z'*num_qubits, 1)])
print(observable)

1.0 * ZZZ


In [17]:
# define two layer QNN
qnn3 = TwoLayerQNN(num_qubits, fm, var_form, observable, qi_sv)

In [18]:
# define (random) input and weights
input3 = np.random.rand(qnn3.num_inputs)
weights3 = np.random.rand(qnn3.num_weights)

In [19]:
# QNN forward pass
qnn3.forward(input3, weights3)

array(-0.61511137)

In [20]:
# QNN backward pass
qnn3.backward(input3, weights3)

(array([-1.48076126, -2.37762168, -0.18183531]),
 array([-0.01749486,  0.27509667, -0.41116957, -0.3181454 , -0.0307983 ,
         0.15724138]))

## 4. `CircuitQNN`

The `CircuitQNN` is just based on a (parametrized) `QuantumCircuit`. This can take input as well as weight parameters and produces samples from the measurement. The samples can either be interpreted as probabilities or directly as a batch of binary output. In the case of probabilities, gradients can be estimated efficiently and the `CircuitQNN` provides a backward pass as well. In case of samples, differentiation is not possible and the backward pass returns `(None, None)`.

Further, the `CircuitQNN` allows to specify different `interpret` options for the measured samples:
- 'int': interprets returns the measured bitstrings as integers
- 'str': returns the measured bitstrings
- 'tuple': returns the measured bitstrings as tuples
- a `callable`: applies a callable to the tuple of binaries and returns the result

The probabilities are then aggregated accordingly.

In case of `interpret='int'` or if `interpret` is set to a callable that returns non-negative integers, the `CircuitQNN` can be configured to return a dense instead of a sparse result.

<font color="red">output shape / dense / return types need to be further discussed</font>

<font color="red">what about `return_samples`?</font>

In [21]:
from qiskit_machine_learning.neural_networks import CircuitQNN

In [22]:
qc = RealAmplitudes(num_qubits, entanglement='linear', reps=1)
print(qc.draw())

     ┌──────────┐     ┌──────────┐            
q_0: ┤ RY(θ[0]) ├──■──┤ RY(θ[3]) ├────────────
     ├──────────┤┌─┴─┐└──────────┘┌──────────┐
q_1: ┤ RY(θ[1]) ├┤ X ├─────■──────┤ RY(θ[4]) ├
     ├──────────┤└───┘   ┌─┴─┐    ├──────────┤
q_2: ┤ RY(θ[2]) ├────────┤ X ├────┤ RY(θ[5]) ├
     └──────────┘        └───┘    └──────────┘


### 4.1 Output: sparse integer probabilities

In [23]:
# specify circuit QNN
qnn4 = CircuitQNN(qc, [], qc.parameters, interpret='int', dense=False, quantum_instance=qi_qasm)

In [24]:
# define (random) input and weights
input4 = np.random.rand(qnn4.num_inputs)
weights4 = np.random.rand(qnn4.num_weights)

In [25]:
# QNN forward pass
qnn4.forward(input4, weights4)

{7: 0.01, 6: 0.01, 1: 0.02, 4: 0.27, 5: 0.01, 2: 0.03, 0: 0.65}

In [26]:
# QNN backward pass
qnn4.backward(input4, weights4)

([],
 [{0: -0.0025057794458004934,
   1: -0.00586966862480692,
   2: -0.010273744832269305,
   3: 0.010567264036027634,
   4: 0.017682995574414107,
   5: -0.020044055365277896,
   6: -0.026548867685693298,
   7: 0.03699185634340618},
  {0: -0.05876201548661364,
   1: -0.0012820440773502004,
   2: 0.0403634704387538,
   3: 0.00249390812591016,
   4: -0.09805120950434351,
   5: -0.0046882592865879445,
   6: 0.11121055630828422,
   7: 0.00871559348194708},
  {0: -0.3964347660835613,
   1: -0.022500053834983904,
   2: -0.014202347163585458,
   3: -0.0003320513514119419,
   4: 0.39037792115032194,
   5: 0.021363879648834013,
   6: 0.020259192096824828,
   7: 0.0014682255375618365},
  {0: -0.15926628337083015,
   1: 0.1592662833708302,
   2: -0.007994779338802402,
   3: 0.007994779338802403,
   4: -0.051770011832395005,
   5: 0.051770011832395005,
   6: -0.00482546054338656,
   7: 0.004825460543386562},
  {0: -0.14644139529898903,
   1: -0.008694937582787096,
   2: 0.14644139529898903,
   3:

### 4.2 Output: sparse string probabilities

In [27]:
# specify circuit QNN
qnn5 = CircuitQNN(qc, [], qc.parameters, interpret='str', dense=False, quantum_instance=qi_qasm)

In [28]:
# define (random) input and weights
input5 = np.random.rand(qnn5.num_inputs)
weights5 = np.random.rand(qnn5.num_weights)

In [29]:
# QNN forward pass
qnn5.forward(input5, weights5)

{'100': 0.09, '001': 0.13, '110': 0.18, '000': 0.45, '111': 0.15}

In [30]:
# QNN backward pass
qnn5.backward(input5, weights5)

([],
 [{'000': -0.2588217873373816,
   '001': 0.10632406220257021,
   '010': -0.001643067514288736,
   '011': -3.3156272785305783e-05,
   '100': -0.029837547735772043,
   '101': 0.004315448276981103,
   '110': -0.14259208274508497,
   '111': 0.32228813112576127},
  {'000': -0.3158697693106967,
   '001': 0.0584502817840377,
   '010': -0.006486278561273241,
   '011': 0.000339791871989388,
   '100': -0.076380354634677,
   '101': 0.006624300211620933,
   '110': 0.31657541605863615,
   '111': 0.016746612580362592},
  {'000': -0.24111538027686724,
   '001': -0.059525751166685244,
   '010': 0.020648539299883993,
   '011': 0.006445036842402697,
   '100': 0.18025932169985298,
   '101': 0.02829790667639321,
   '110': 0.040207519277130344,
   '111': 0.02478280764788931},
  {'000': -0.2261736241658003,
   '001': 0.2261736241658003,
   '010': -0.0010990948832156884,
   '011': 0.0010990948832156884,
   '100': -0.023070672198536573,
   '101': 0.023070672198536573,
   '110': -0.16146953208701448,
   '

### 4.3 Output: dense parity probabilities

In [31]:
# specify circuit QNN
parity = lambda x: np.sum(x) % 2
output_shape = 2  # this is required in case of a callable with dense output
qnn6 = CircuitQNN(qc, [], qc.parameters, interpret=parity, dense=True, output_shape=output_shape,
                  quantum_instance=qi_qasm)

In [32]:
# define (random) input and weights
input6 = np.random.rand(qnn6.num_inputs)
weights6 = np.random.rand(qnn6.num_weights)

In [33]:
# QNN forward pass
qnn6.forward(input6, weights6)

array([[0.55, 0.45]])

In [34]:
# QNN backward pass
qnn6.backward(input6, weights6)

(array([], shape=(1, 2, 0), dtype=float64),
 array([[[-0.3483633 ,  0.12097291, -0.14164533, -0.33666166,
          -0.00650527, -0.08430846],
         [ 0.3483633 , -0.12097291,  0.14164533,  0.33666166,
           0.00650527,  0.08430846]]]))

### 4.4 Output: dense samples

In [35]:
# TODO: show how to setup a circuit QNN that returns samples... (but no gradients)

In [36]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright